In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import datetime
from iautils import cascade

from datetime import datetime
from astropy.coordinates import SkyCoord

/opt/pysetup/.venv/lib/python3.11/site-packages/chime_frb_api/__init__.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution as _get_distribution


In [2]:
sys.path.append("/arc/home/mseth/.local/lib/python3.11/site-packages")
import memory_profiler

In [3]:

%load_ext memory_profiler

In [ ]:
sys.path.insert(0, os.path.abspath('beam-model'))
from beam_model import utils, formed

In [ ]:
freqs = np.linspace(400.390625, 800, 256)            # The data has 256 frequency channels
has = np.array(np.linspace(-105, 104.90278, 1024))   # 1024 HAs

source_name = "CYG_A"
coords = SkyCoord.from_name(source_name)
source_ra = coords.ra.deg
source_dec = coords.dec.deg


# BF to JY conversion equation
def bf_to_jy(bf_spectrum, f_good):
    factor = (np.square(1024) * 128) / (np.square(4) * 0.806745 * 400)
    result = bf_spectrum / ( factor * np.square(f_good) ) 
    return result

Data

Stuff we define beforehand

In [24]:
filepaths = []
dms = []
cascade_objs = []
path = "/arc/projects/chime_frb/mseth/tau_A_data/initial_try"

for (root, dirs, file) in os.walk(path):
    for f in file: 
        filepaths.append(os.path.join(root, f))

In [ ]:
%%memit
dms = []
max_tidxs = []
max_timestamps = []

for filepath in filepaths:
    try:
        cascade_obj = cascade.load_cascade_from_file(filepath)
        cascade_obj.dm=57.70 
        dm = cascade_obj.best_snr_dm
        
        if dm >= 55 or dm <= 57:
            ts = np.nansum(cascade_obj.beams[cascade_obj._max_beam_idx].intensity, axis=0)
            max_tidx = np.argmax(ts)
            max_timestamp = cascade_obj.event_time + pd.Timedelta(seconds = max_tidx / 1000)

            max_tidxs.append(max_tidx)
            max_timestamps.append(max_timestamp)
            dms.append(dm)
            del cascade_obj
        else:
            del cascade_obj
        
    except Exception as e:
        print(e)
        continue

Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
De-dispersing to 57.7 pc cm-3..
Using 400.1953125 MHz as reference..
Padding shifted channels using median value..

De-dispersing to 57.7 pc cm-3..
Using 400.1953125 MHz as reference..
Padding shifted channels using median value..

De-dispersing to 57.7 pc cm-3..
Using 400.1953125 MHz as reference..
Padding shifted channels using median value..

De-dispersing to 57.7 pc cm-3..
Using 400.1953125 MHz as reference..
Padding shifted channels using median value..

De-dispersing to 57.7 pc cm-3..
Using 400.1953125 MHz as reference..
Padding shifted channels using median value..

De-dispersin

In [ ]:
%%memit
from iautils import cascade

path = "/arc/projects/chime_frb/mseth/tau_A_data/initial_try"

filepaths = []
dms = []
cascade_objs = []

for (root, dirs, file) in os.walk(path):
    for f in file: 
        filepaths.append(os.path.join(root, f))
        
for filepath in filepaths:
        # Load the cascade file
    try:
        cascade_obj = cascade.load_cascade_from_file(filepath)
        #cascade_obj.dm=57.70 ## De-dispersing at 57.70
    except Exception as e:
        print(e)
        continue
    #dm = cascade_obj.best_snr_dm
    #dms.append(dm)
    cascade_objs.append(cascade_obj)

/opt/pysetup/.venv/lib/python3.11/site-packages/chime_frb_api/__init__.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution as _get_distribution


Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing.

In [ ]:
dms

In [ ]:
path = "/arc/projects/chime_frb/mseth/tau_A_data"

filepaths = []
dms = []
cascade_objs = []

for (root, dirs, file) in os.walk(path):
    for f in file: 
        filepaths.append(os.path.join(root, f))

In [ ]:
## DATA ## 
max_tidxs = []             # Time index at peak
max_timestamps = []        # Timestamp of peak
spectra_at_peak = []       # Spectrum at peak time
beam_ids = []              # Beam ID that detected peak
has_at_peak = []           # HA of peak detection

for i, cascade_obj in enumerate(cascade_objs):
    ts = np.nansum(cascade_objs[i].beams[cascade_objs[i]._max_beam_idx].intensity, axis=0)
    
    max_tidx = np.argmax(ts)
    max_timestamp = cascade_objs[i].event_time + pd.Timedelta(seconds = max_tidx / 1000)
    spectrum_at_peak = cascade_objs[i].beams[cascade_objs[i]._max_beam_idx].intensity[:, max_tidx]
    beam_id = cascade_obj.beams[cascade_obj.__max_beam_id].beam_no

    x, y = utils.get_position_from_equatorial(source_ra, source_dec, max_timestamp)
    
    max_tidxs.append(max_tidx)
    max_timestamps.append(max_timestamp)
    spectra_at_peak.append(spectrum_at_peak)
    has_at_peak.append(x)

In [ ]:
plt.figure()
plt.plot(freqs, spectra_at_peak[0])
plt.title("Example of uncorrected data")
plt.show()

Beams & Correcting

In [ ]:
## PRIMARY BEAM RESPONSE ##

# Load in holography data 
path = "/users/mseth/holography_data/npz_files"

npz_files = []
for (root, dirs, file) in os.walk(path):
    for f in file: 
        npz_files.append(os.path.join(root, f))
xx_list = []
yy_list = []
for file in npz_files:
    data = np.load(file)
    xx = data['XX']
    yy = data['YY']
    xx_list.append(xx)
    yy_list.append(yy)

xx = np.vstack(xx_list)
yy = np.vstack(yy_list)
intensity = xx + yy / 2

# Get spectra at every peak HA 
presponse_norm = []

for ha in has_at_peak:
    idx = (np.abs(has - ha)).argmin()         # Find index of closest HA to HA at peak
    xx_response = xx[:,idx]
    yy_response = yy[:, idx]
    
    int_response = intensity[:,idx]           # Get intensity spectrum at that HA 
    int_masked = intensity[:,1760:1840]       # Get maximum intensity in the main beam
    inty_max = np.max(intensity_masked)       
    
    int_norm = int_response / intensity_max   # Normalize intensity spectrum at the HA
    presponse_norm.append(int_norm)

In [ ]:
plt.figure()
plt.plot(np.linspace(400.390625, 800, 1024), np.log10(int_response[0]))
plt.title("Example of primary beam response")
plt.show()

In [ ]:
## CORRECTING FOR PRIMARY BEAM RESPONSE ##

p_corrected_sp = []

for i, (spectra, presponse) in enumerate(zip(spectra_at_peak, presponse_norm)):
    presponse_ds = int_response[::4]                             # Downsample primary response spectrum from 1024 freq channels to 256 (matching data)
    spectra_at_peak_corrected = spectra_at_peak / presponse_ds   # Correction
    p_corrected_sp.append(spectra_at_peak_corrected)

In [ ]:
## SYNTHESIZED BEAM RESPONSE + CORRECTING

sbm = formed.FFTFormedBeamModel()

calibrated_spectra = []

for i, (beam_id, ha, y, spectrum) in enumerate(zip(beam_ids, has_at_peak, y_at_peak, p_corrected_sp)):
    sensitivity = sbm.get_sensitivity([beam_id], np.array([ha, y_at_peak]).T, freqs).squeeze()
    
    mean_sensitivity = np.nanmean(sensitivity, axis=1)            # Average sensitivity across frequencies
    peak_sensitivity_ind = np.argmax(mean_sensitivity)            
    sensitivity_at_peak = sensitivity[peak_sensitivity_ind, :]    # Spectrum of synthesized response at its peak

    spectra_corrected = spectrum / sensitivity_at_peak            # Correction
    spectra_jy = bf_to_jy(spectra_corrected, 1)                   # Conversion
    calibrated_spectra.append(spectra_jy)